[Look Up](https://www.luogu.org/problemnew/show/P2947)。给定一数组，求各数字右边第一个比该数字大的数，没有则设置0.

思路：单调栈的典型应用。从后往前扫描数组，设立一个栈，栈中始终保存比当前数字大的元素，若有小的全部弹出。

In [ ]:
def LookUp(nums):
    n=len(nums)
    res=[0]*n
    s=list()
    
    for idx in range(n-1,-1,-1):
        # 先排空栈中不大于当前数的所有数字
        while s and nums[s[-1]]<=nums[idx]:
            s.pop()
        
        # 栈中剩下的元素都是比当前大的数了
        if not s:    # 栈为空
            res[idx]=0
        else:    # 栈不空，比当前大的最近的数就是栈顶元素
            res[idx]=s[-1]
        
        s.append(idx)
        
    return res

[Remove K Digits](https://leetcode.com/problems/remove-k-digits/)。给一个数字字串，删掉其中的$k$位，使得得到的数字最小。

思路：单调栈。所有位顺序入栈，当某一位小于栈顶元素时，需要弹栈。注意弹栈的次数不能超过$k$次。不难，实现逻辑稍微复杂一点。

In [ ]:
def removeKdigits(num: str, k: int) -> str:
    n = len(num)
    if k >= n:
        return '0'

    s = list()
    cnt = 0

    for ch in num:
        if cnt < k:
            if not s:
                s.append(ch)
                continue

            while s and int(ch) < int(s[-1]) and cnt < k:
                s.pop()
                cnt += 1
            s.append(ch)

        else:
            s.append(ch)

    return str(int(''.join(s[:n-k])))

[Next Greater Element I](https://leetcode.com/problems/next-greater-element-i/)。给一不含重复元素的数组$nums2$，选出一个子集$nums1$，求$nums1$中所有数字在$nums2$中最近的比它大的数字。

思路：求出每一个数字的最近比它大的数字很简单，单调栈即可解决。如果是一组数，使用哈希表记录即可。

In [6]:
def nextGreaterElement(nums1, nums2):
    table = dict()
    s = list()

    for num in nums2:
        if not s:
            s.append(num)

        while s and num > s[-1]:
            pre_num = s.pop()
            table[pre_num] = num

        s.append(num)

    while s:
        pre_num = s.pop()
        table[pre_num] = -1

    return [table[num] for num in nums1]

[-1, 3, -1]

[Next Greater Element II](https://leetcode.com/problems/next-greater-element-ii/)。

思路：

In [12]:
def nextGreaterElements(nums):
    double_nums = nums+nums
    s = list()
    table = dict()

    for num in double_nums:
        if not s:
            s.append(num)
            continue

        while s and num > s[-1]:
            pre_num = s.pop()
            if pre_num not in table.keys():
                table[pre_num] = num

        s.append(num)

    while s:
        pre_num = s.pop()
        if pre_num not in table.keys():
            table[pre_num] = -1

    return [table[num] for num in nums]

nextGreaterElements([100,1,11,1,120,111,123,1,-1,-100])

[120, 11, 120, 11, 123, 123, -1, 11, 100, 100]